## Our Mission ##

在前面课程的练习中，你已经用朴素贝叶斯算法对这个[数据集](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) 进行过垃圾邮件分类。在这个 notebook 里，我们将使用刚学到的一些新技术，在前面分析的基础上再做一些扩展。


> 我们先快速地重做前面的朴素贝叶斯垃圾邮件分类 notebook 中的工作。我们提供了以前工作区（workspace）的基本代码，请运行下面的单元格。

In [1]:
# Import our libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Read in our dataset
df = pd.read_table('smsspamcollection/SMSSpamCollection',
                   sep='\t', 
                   header=None, 
                   names=['label', 'sms_message'])

# Fix our response value
df['label'] = df.label.map({'ham':0, 'spam':1})

# Split our dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

# Instantiate our model
naive_bayes = MultinomialNB()

# Fit our model to the training data
naive_bayes.fit(training_data, y_train)

# Predict on the test data
predictions = naive_bayes.predict(testing_data)

# Score our model
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

/home/leon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: read_table is deprecated, use read_csv instead.
  del sys.path[0]


Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439562


### Turns Out...

从上面的分数可以看出，朴素贝叶斯模型实际上对垃圾邮件和正常邮件进行了有效分类。不过，让我们看一些其他模型，看是否还可以进一步改善。

具体地说， 我们将介绍以下技术:

* [BaggingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)
* [RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)
* [AdaBoostClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier)

这是另一个非常有用的集成算法指南 [文档](http://scikit-learn.org/stable/modules/ensemble.html) 。

这些集成算法是你在本单元中看到的技术的组合：

* 对一个学习器的数据**用自助采样法抽取数据**  (bagging)。
* 对一个学习器的数据 **抽取属性 ** (与bagging相结合来表示随机森林的两个随机成分)。
* 把各有所长的弱学习器**集成**为一个强学习器  (boosting) 。


在本notebook中，让我们对这些方法做一些练习，这将帮助你熟悉在python环境中进行监督学习的一般过程。

因为在前一个notebook中，我们已清理和矢量化了文本，这里我们可以专注于有趣的部分 – 机器学习部分。

### This Process Looks Familiar...

如果你想应用一个监督学习方法，通常有以下5个步骤 (你在上面用过的)：

1. **导入** 模型。
2. 用感兴趣的超参数**实例化**模型。
3. 用模型**拟合**训练数据。
4. 用模型**预测**测试数据。
5. 通过比较预测值和真实值对模型进行**评分**。

按照本notebook中的步骤，对每个集成方法分别执行以下步骤：**BaggingClassifier**， **RandomForestClassifier** 和 **AdaBoostClassifier**。

> **步骤 1：**首先根据这个文档导入（`import`）所有三个模型。

In [2]:
# Import the Bagging, RandomForest, and AdaBoost Classifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier

> **步骤2：**我们已经导入了所有分类器，现在根据注释中提供的超参数实例化（`instantiate`）这些模型。在后面的课程中，我们将介绍如何自动化查找最佳的超参数的过程。现在，让我们先熟悉这个流程和新算法。

In [3]:
# Instantiate a BaggingClassifier with:
# 200 weak learners (n_estimators) and everything else as default values
bag_mod = BaggingClassifier(n_estimators=200)


# Instantiate a RandomForestClassifier with:
# 200 weak learners (n_estimators) and everything else as default values
rf_mod = RandomForestClassifier(n_estimators=200)

# Instantiate an a AdaBoostClassifier with:
# With 300 weak learners (n_estimators) and a learning_rate of 0.2

ada_mod = AdaBoostClassifier(n_estimators=300,learning_rate=0.2)

> **步骤3：**你已经实例化了所有模型，现在开始用 **training_data** 和 **y_train** 来拟合（`fit`）这些模型。这可能需要一点时间，毕竟你要拟合700个弱学习器！

In [4]:
# Fit your BaggingClassifier to the training data
bag_mod.fit(training_data, y_train)

# Fit your RandomForestClassifier to the training data
rf_mod.fit(training_data, y_train)

# Fit your AdaBoostClassifier to the training data
ada_mod.fit(training_data, y_train)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.2,
                   n_estimators=300, random_state=None)

> **步骤4：**你已经拟合好每个模型，现在开始对用每个模型预测（`predict`）**testing_data**。

In [5]:
# Predict using BaggingClassifier on the test data
bag_preds = bag_mod.predict(testing_data) 

# Predict using RandomForestClassifier on the test data
rf_preds = rf_mod.predict(testing_data)

# Predict using AdaBoostClassifier on the test data
ada_preds = ada_mod.predict(testing_data)


> **步骤5：**现在你已经完成了预测，对每个模型使用下面的函数，将预测值与真实值进行比较 - 这将会为每个模型的表现打分（`score` ）。在这里再次显示朴素贝叶斯模型也很好，这样我们就可以将它们并排进行比较。

In [6]:
def print_metrics(y_true, preds, model_name=None):
    '''
    INPUT:
    y_true - the y values that are actually true in the dataset (NumPy array or pandas series)
    preds - the predictions for those values from some model (NumPy array or pandas series)
    model_name - (str - optional) a name associated with the model if you would like to add it to the print statements 
    
    OUTPUT:
    None - prints the accuracy, precision, recall, and F1 score
    '''
    if model_name == None:
        print('Accuracy score: ', format(accuracy_score(y_true, preds)))
        print('Precision score: ', format(precision_score(y_true, preds)))
        print('Recall score: ', format(recall_score(y_true, preds)))
        print('F1 score: ', format(f1_score(y_true, preds)))
        print('\n\n')
    
    else:
        print('Accuracy score for ' + model_name + ' :' , format(accuracy_score(y_true, preds)))
        print('Precision score ' + model_name + ' :', format(precision_score(y_true, preds)))
        print('Recall score ' + model_name + ' :', format(recall_score(y_true, preds)))
        print('F1 score ' + model_name + ' :', format(f1_score(y_true, preds)))
        print('\n\n')

In [7]:
# Print Bagging scores
print_metrics(y_test, bag_preds, 'bagging')

# Print Random Forest scores
print_metrics(y_test, rf_preds, 'random forest')

# Print AdaBoost scores
print_metrics(y_test, ada_preds, 'adaboost')

# Naive Bayes Classifier scores
print_metrics(y_test, predictions, 'naive bayes')


Accuracy score for bagging : 0.9741564967695621
Precision score bagging : 0.9116022099447514
Recall score bagging : 0.8918918918918919
F1 score bagging : 0.9016393442622951



Accuracy score for random forest : 0.9820531227566404
Precision score random forest : 1.0
Recall score random forest : 0.8648648648648649
F1 score random forest : 0.927536231884058



Accuracy score for adaboost : 0.9770279971284996
Precision score adaboost : 0.9693251533742331
Recall score adaboost : 0.8540540540540541
F1 score adaboost : 0.9080459770114943



Accuracy score for naive bayes : 0.9885139985642498
Precision score naive bayes : 0.9720670391061452
Recall score naive bayes : 0.9405405405405406
F1 score naive bayes : 0.9560439560439562





### Recap

现在你已经看到了一些集成模型的整个工作流程！ 

1. **导入** 模型。
2. 用感兴趣的超参数**实例化**模型。
3. 用模型**拟合**训练数据。
4. **预测**测试数据。
5. 通过比较预测值和真实值对模型进行**评分**。

搞定！这是机器学习的一个非常常见的过程。


### But, Wait...

你可能会问 - 

* 这些指标是什么意思？ 

*如何优化以获得最佳模型？  

* 每个模型都有这么多超参数，我如何确定每个模型的最佳值？

**这些问题正是本课程最后两节课的全部内容。**

**注意，你可以通过单击左上角的橙色图标来获得此notebook的解决方案！ **
